In [1]:
# Parameter set by me, I can change this as desired and the rest of the model will work fine
numStudents = 101 # TODO: Make more realistic?

101

In [2]:
import Pkg; 
Pkg.add("GLPK")
Pkg.add("JuMP")
Pkg.add("CSV")
Pkg.add("DataFrames")

In [3]:
const DATA_DIR = joinpath(@__DIR__, "data");


import DataFrames
import CSV
import Random
import GLPK
using JuMP

In [4]:
########### Parsing the CSV data: #######################

csv_df = CSV.read(joinpath(DATA_DIR, "courseData.csv"), DataFrames.DataFrame)

ProfsSet = Set{String}(skipmissing(csv_df[!, 1])) # set of all profs
numProfs = length(ProfsSet)
CoursesSet = Set{String}() # set of all courses
profToCourseDict = Dict{String, Set{String}}() # Dict of the format {ProfName => {ProfCourse1, ProfCourse2, ...}}

for i in 1:7 # The range 1:7 is based on the current format of the csv
    currProf = csv_df[i, 1]
    currCourses = Set{String}()
    for j in 2:4
        if ~ ismissing(csv_df[i, j])
            push!(currCourses, csv_df[i, j])
            push!(CoursesSet, csv_df[i, j] )
        end        
    end
    profToCourseDict[currProf] = currCourses
end

numCourses = length(CoursesSet) # Based on number of courses in csv

############ Getting Course Timings ##############

courseTimingsDict = Dict{String, Vector{Any}}()

for i in 1:numProfs
    currProf = csv_df[i, 1]
    days = csv_df[i, 5]
    startTime = csv_df[i, 6]
    EndTime = csv_df[i, 7]
    for course in profToCourseDict[currProf]
        courseTimingsDict[course] = [days, startTime, EndTime]
    end    
end

# courseTimingsDict

####### Cap on Number of Seats in a Course ########

# TODO: THIS WILL CAUSE THE PROGRAM TO FAVOUR COURSES THAT HAVE A HIGHER NUMBER OF SEATS
# SHOULD I FACTOR THIS INTO THE PROGRAM SOMEHOW? (Maybe each prof can offer a max number of seats? I'm not sure yet)

courseCapDict = Dict{String, Int32}()

for i in 1:numCourses
    courseCapDict[csv_df[i,9]] = csv_df[i,10]
end

# courseCapDict


#############################################################

In [5]:
############ Helper Function ####################################


# Function that returns a random list of integers of length "len" with the sum of the integers equal to
# "fixedSum" to reflect how a student distributes a fixed number of points among the courses being offered
function getRandomFixedSum(len, fixedSum)
    res = Int64[]
    currSum = 0
    
    # adding a random integer for each course in the list
    for i in 1:len
        # if fixedSum reached, the rest of the  values will be 0
        if currSum >= fixedSum
            push!(res, 0)
        else
            # this if condition ensures that each student uses all the points available to him
            # by spending all remaining points on the last course in the list
            if i == len
                push!(res, (fixedSum-currSum))
            else
                currRand = rand(0:(fixedSum-currSum)) # ensures student does not spend more points than they have left
                push!(res, currRand)
                currSum += currRand
            end
        end        
    end
    
    # shuffling the list to ensure no bias towards any specific course (will need to make this more reflective 
    # of reality later)
    Random.shuffle!(res)
    
    # Since 0 represents that a student does not want to take a course, this discourages a student being assigned to
    # courses that they do not want
    replace!(res, 0 => -500)
    return res
end 

#############################################################

getRandomFixedSum (generic function with 1 method)

In [6]:
############ Creating Course Rating Dictionary ##############

# courseRatingMatrix = zeros(Int32, numStudents, numCourses) # Might switch to using matrices instead of dicts at some point.
courseRatingDict = Dict{Tuple{Int64, String}, Int64}() # NOTE: Might need to change the type of the key when the students become strings instead of numbers

for i in 1:numStudents
    currRatings = getRandomFixedSum(numCourses,10)
    for (index, course) in enumerate(CoursesSet)
        courseRatingDict[(i, course)] = currRatings[index]
    end
#     courseRatingMatrix[i, :] = getRandomFixedSum(numCourses,10) # Might switch to using matrices instead of dicts at some point.
end

# courseRatingMatrix
# courseRatingDict

#############################################################

In [7]:
##### Creating Course Timings Dictionary #######

# courseTimeBool is a dictionary of the format:
# { (course, timestep) => 1 if course takes place at timestep and 0 otherwise}
# An example of a timestep is W1335 which refers to Wednesday at 1:35pm


courseTimeBool = Dict{Tuple{String, String}, Int32}()

discreteTimes = []
TimeIdsSet = Set()
for k in 7:23
    for j in 0:5
        for i in [0, 5]
            currTime = "$k$(j)$(i)"
            currTime = parse(Int64, currTime)
            push!(discreteTimes, currTime)
        end
    end
end

days = "MTWRF"
for currTime in discreteTimes
    for day in days
        currTimeString = string(currTime)
        currTimeId = "$(day)$(currTimeString)"
        push!(TimeIdsSet, currTimeId)
        for course in CoursesSet
            if day in courseTimingsDict[course][1]
                startTime = courseTimingsDict[course][2]
                endTime = courseTimingsDict[course][3]
                # Not strictly greater than or less than because
                # courses can take place back to back.
                afterStartTime = currTime > startTime
                beforeEndTime = currTime < endTime
                
                if afterStartTime & beforeEndTime
                    courseTimeBool[(course, currTimeId)] = 1
                end
            end
        end
    end
end

# For all the keys that did not get a value of 1, I'm setting them
# to 0
# This is essentially what a defaultdict does, but defaultdicts
# don't work well with optimizers so I'm doing this "manually" here instead
for timeId in TimeIdsSet
    for course in CoursesSet
        if ~ ((course, timeId) in keys(courseTimeBool))
            courseTimeBool[(course, timeId)] = 0
        end
    end
end

# courseTimeBool

#############################################################

In [8]:
model = Model(GLPK.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [9]:
########## DECISION VARIABLES ########################

@variable(model, coursesBool[course in CoursesSet], Bin);

@variable(model, studentAssignments[student in 1:numStudents, course in CoursesSet], Bin);

####### Trick for Linearizing Quadratic Terms!!! ###############

# The essense of the trick is it ensures that : z[student, course] = studentAssignment[student, course] * coursesBool[course]
# using the 4 constraints below
# got the trick from this website: https://orinanobworld.blogspot.com/2010/10/binary-variables-and-quadratic-terms.html

@variable(model, z[student in 1:numStudents, course in CoursesSet], Bin);

for student in 1:numStudents
    for course in CoursesSet
        @constraint(model, z[student, course] <= studentAssignments[student, course])
        @constraint(model, z[student, course] >= 0) # this isnt really needed in our case
        @constraint(model, z[student, course] <= coursesBool[course])
        @constraint(model, z[student, course] >= (coursesBool[course] - (1-studentAssignments[student, course])))
    end
end

#################################################################

In [10]:
###### CONSTRAINTS #############################################

# Constraint: min and max number of electives given to a student
for student in 1:numStudents
    @constraint(model, 1 <= sum(z[student, course] for course in CoursesSet) <= 2)
end

# Constraint: Cap on # of students in a course
for course in CoursesSet
    @constraint(model, sum(z[student, course] for student in 1:numStudents) <= courseCapDict[course]) 
end


# Constraint: Time Conflict constraint for Students
for timeId in TimeIdsSet
    for student in 1:numStudents
        @constraint(model, sum([courseTimeBool[(course, timeId)]*z[student, course] for course in CoursesSet]) <= 1)
    end
end

# Constraint: Time Conflict constraint for Profs (Uncomment when Profs teach courses at different times) 
# (currently each prof teaches only one course so this isnt needed.)

# for timeId in TimeIdsSet
#     for prof in ProfsSet
#         @constraint(model, sum([courseTimeBool[(course, timeId)] for course in profToCourseDict[prof]]) <= 1)
#     end
# end

# Contraint: at most 6 courses to be chosen (replaced in favour of the next constraint)
# @constraint(model, sum([coursesBool[course] for course in CoursesSet]) <= 6)

# Constraint: exactly one course from one prof
for prof in ProfsSet
    @constraint(model, sum([coursesBool[course] for course in profToCourseDict[prof]]) == 1)
end

#################################################################

In [11]:
# Objective
@objective(model, Max, sum([z[student, course]*courseRatingDict[student, course] for student in 1:numStudents, course in CoursesSet]));

In [12]:
optimize!(model)

In [13]:
solution_summary(model)

* Solver : GLPK

* Status
  Termination status : OPTIMAL
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Message from the solver:
  "Solution is optimal"

* Candidate solution
  Objective value      : -9110.0
  Objective bound      : -9110.0

* Work counters
  Solve time (sec)   : 328.94000


In [14]:
### DISPLAYING THE RESULTS ##############

for course in CoursesSet
    if getvalue(coursesBool[course]) > 0.5 # tol of 0.5
        println("Students assigned to $(course) are:")
        currStudents = []
        for student in 1:numStudents
            if getvalue(studentAssignments[student, course]) > 0.5 # tol of 0.5
                push!(currStudents, student)
            end
        end
        println(currStudents)
    end
end

#################################################################

Students assigned to B1 are:
Any[17, 19, 62, 79, 85, 86, 88, 89, 93, 100]
Students assigned to C1 are:
Any[5, 23, 34, 42, 57, 63, 68, 70, 74, 99]
Students assigned to F1 are:
Any[25, 30, 31, 36, 40, 46, 48, 50, 53, 58, 69, 71, 72, 75, 79, 82, 84, 87, 90, 98]
Students assigned to G1 are:
Any[2, 4, 7, 12, 26, 32, 43, 52, 53, 54, 58, 59, 60, 66, 74, 76, 81]
Students assigned to E2 are:
Any[10, 12, 13, 16, 18, 20, 24, 35, 37, 41, 43, 47, 55, 61, 64, 67, 91, 94, 96, 100]
Students assigned to D3 are:
Any[1, 9, 11, 14, 15, 21, 22, 27, 30, 33, 51, 56, 73, 75, 77, 78, 83, 84, 91, 101]
Students assigned to A1 are:
Any[3, 6, 8, 28, 29, 38, 39, 44, 45, 49, 65, 80, 92, 95, 97]


In [15]:
### DISPLAYING THE RESULTS ##############

for student in 1:numStudents
    currCourses = String[]
    for course in CoursesSet
        if getvalue(coursesBool[course]) > 0.5 # tol of 0.5
            if getvalue(studentAssignments[student, course]) > 0.5 # tol of 0.5
                push!(currCourses, course)
            end
        end
    end
    println("Student $(student): $(currCourses)")
end

#################################################################

Student 1: ["D3"]
Student 2: ["G1"]
Student 3: ["A1"]
Student 4: ["G1"]
Student 5: ["C1"]
Student 6: ["A1"]
Student 7: ["G1"]
Student 8: ["A1"]
Student 9: ["D3"]
Student 10: ["E2"]
Student 11: ["D3"]
Student 12: ["G1", "E2"]
Student 13: ["E2"]
Student 14: ["D3"]
Student 15: ["D3"]
Student 16: ["E2"]
Student 17: ["B1"]
Student 18: ["E2"]
Student 19: ["B1"]
Student 20: ["E2"]
Student 21: ["D3"]
Student 22: ["D3"]
Student 23: ["C1"]
Student 24: ["E2"]
Student 25: ["F1"]
Student 26: ["G1"]
Student 27: ["D3"]
Student 28: ["A1"]
Student 29: ["A1"]
Student 30: ["F1", "D3"]
Student 31: ["F1"]
Student 32: ["G1"]
Student 33: ["D3"]
Student 34: ["C1"]
Student 35: ["E2"]
Student 36: ["F1"]
Student 37: ["E2"]
Student 38: ["A1"]
Student 39: ["A1"]
Student 40: ["F1"]
Student 41: ["E2"]
Student 42: ["C1"]
Student 43: ["G1", "E2"]
Student 44: ["A1"]
Student 45: ["A1"]
Student 46: ["F1"]
Student 47: ["E2"]
Student 48: ["F1"]
Student 49: ["A1"]
Student 50: ["F1"]
Student 51: ["D3"]
Student 52: ["G1"]
Stu